In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

In [ ]:
from tkinter import filedialog
from tkinter import Tk
import os

root = Tk()
fpath = filedialog.askopenfilename(initialdir = os.getcwd(), title = 'Please select a file:')

root.destroy()

assert fpath[fpath.rindex('/') + 1:].find('.') == -1

In [ ]:
import pickle

with open(fpath,'rb') as f:

    locations_new = pickle.load(f)
    addresses = pickle.load(f)
    distances = pickle.load(f)
    routes = pickle.load(f)
    north_bound = pickle.load(f)
    south_bound = pickle.load(f)
    left_bound = pickle.load(f)
    right_bound = pickle.load(f)
    nc = pickle.load(f)
    nr = pickle.load(f)
    service_time_customer = pickle.load(f)
    service_time_rs = pickle.load(f)
    v = pickle.load(f)
    r = pickle.load(f)
    L = pickle.load(f)
    T = pickle.load(f)

In [ ]:
nd = 1
nr = nr
rd = 8 #4*2
rr = 4 #2*2

D = [i for i in range(0, nd * rd)] # set of dummy depots
C = [i for i in range(nd * rd, nd * rd + nc)] # set of customers
R = [i for i in range(nd * rd + nc, nd * rd + nc + nr * rr)] # set of dummy rs's
id_R = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] # identity of rs in set R

V = D + C + R # set of all vertices

print(D)
print(C)
print(R)

In [ ]:
dictionary_map = {} # index_old -> index_new

d_list = [i for i in range(0, rd)]
dictionary_map[0] = d_list

for i in range(0, nc):
    dictionary_map[1 + i] = [rd + i]

for i in range(0, nr):
    r_list = []
    for j in range(nd * rd + nc + i * rr, nd * rd + nc + (i + 1) * rr):
        r_list.append(j)
    dictionary_map[1 + nc + i] = r_list
    
print(dictionary_map)

In [ ]:
costs = {}

for (i, j) in distances:
    for m in dictionary_map[i]:
        for n in dictionary_map[j]:
            costs[(m, n)] = distances[(i, j)]
            
#print(costs)

In [ ]:
routes_mapped = {}

for (i, j) in routes:
    for m in dictionary_map[i]:
        for n in dictionary_map[j]:
            routes_mapped[(m, n)] = routes[(i, j)]
            
#print(routes_mapped.keys())

In [ ]:
D_in = [i for i in range(0, nd * rd, 2)] # ingoing dummy depots(even vertices)
D_out = [i for i in range(1, nd * rd, 2)] # outgoing dummy depots(odd vertices)

R_in = [i for i in range(nd * rd + nc, nd * rd + nc + nr * rr, 2)] # ingoing dummy rs's
R_out = [i for i in range(nd * rd + nc + 1, nd * rd + nc + nr * rr + 1, 2)] # outgoing dummy rs's

print(D_in)
print(D_out)
print(R_in)
print(R_out)

In [ ]:
E_D = [(D_in[i], D_out[i]) for i in range(0,len(D_in))] # internal connections within D
E_R = [(R_in[i], R_out[i]) for i in range(0,len(R_in))] # internal connections within R

print(E_D)
print(E_R)

In [ ]:
E_rest =   [(i,j) for i in D_out for j in C + R_in] \
         + [(i,j) for i in C for j in D_in + C + R_in if i != j] \
         + [(i,j) for i in R_out for j in D_in + C] \
         + [(i,j) for i in R_out for j in R_in if id_R[R.index(i)] != id_R[R.index(j)]]

# line1 descreibes outgoing connections from D_out
# line2 descreibes outgoing connections from C and internal connections within C, note cc' and c'c are different tours
# line3 descreibes outgoing connections from R_out to D_in or to C
# line4 enables rs to rs' interconnections

E = E_rest + E_D + E_R # set of all edges

#print(E)

In [ ]:
from docplex.mp.model import Model

mdl = Model('GVRP')

x = mdl.binary_var_dict(E, name = 'x') # binary varible x, x_ij=1 if arc(or edge)_ij is activated
l = mdl.continuous_var_dict(V, name = 'l') # battery-level l, default lower bound = 0
t = mdl.continuous_var_dict(V, name = 't', ub = T) # time t, upper bound = T

mdl.minimize(mdl.sum(costs[i, j] * x[i, j] for i, j in E)) # objective

# fully recharged at depot
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], l[j] == L) for (i, j) in E_D)
# time reset at depot
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], t[j] == 0) for (i, j) in E_D)

# fully recharged at rs
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], l[j] == L) for (i, j) in E_R)
# charging time at rs
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], t[j] == t[i] + service_time_rs) for (i, j) in E_R)

# battery consumption
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], l[j] == l[i] - r * costs[i, j]) for (i, j) in E_rest)
# time flows
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], t[j] == t[i] + costs[i, j] / v) for (i, j) in E_rest if i not in C) 
# time flows + service time at customer
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], t[j] == t[i] + costs[i, j] / v + service_time_customer) for (i, j) in E_rest if i in C) 

for ic in C: # flow conservation at c
    mdl.add_constraint(mdl.sum(x[i, j] for (i, j) in E if j == ic) == 1)
    #mdl.add_constraint(mdl.sum(x[j, k] for (j, k) in E if j == ic) == 1)

for iv in V: # overall flow conservation
    mdl.add_constraint(mdl.sum(x[i, j] for (i, j) in E if j == iv) == mdl.sum(x[j, k] for (j, k) in E if j == iv))

mdl.print_information()

In [ ]:
mdl.set_time_limit(100)

solution = mdl.solve(log_output = True)

In [ ]:
print(solution.solve_status)

In [ ]:
E_active = [i for i in E if x[i].solution_value == 1]

# search for tours in E_active

tours = []

for i in E_active:
    if i[0] in D_out:
        break
tour = list(i)
vi = i[1]
ne = 1 # number of edges counted
Dout = [i[0]]

while ne < len(E_active):
    while not vi in D_in:
        for i in E_active:
            if i[0] == vi:
                break
        vi = i[1]
        tour.append(vi)
        ne = ne + 1
    tours.append(tour)
    ne = ne + 1
    for i in E_active:
        if i[0] in D_out and not i[0] in Dout:
            break
    tour = list(i)
    vi = i[1]
    ne = ne + 1
    Dout.append(i[0])
    
print(tours)

In [ ]:
def get_key (dic, value):
    for key, val in dic.items():
        if value in val:
            return key

tours_demapped = []

for i in tours:
    vi_last = -1
    tour = []
    for j in i:
        vi = get_key(dictionary_map ,j)
        if vi != vi_last:
            tour.append(vi)
            vi_last = vi
    tours_demapped.append(tour)

for i in tours_demapped:
    if i[1] > i[-2]:
        i.reverse()
        
from operator import itemgetter

tours_demapped = sorted(tours_demapped, key = itemgetter(1))
    
print(tours_demapped)

In [ ]:
tour_distances = []

for i in tours:
    distance = 0
    for j in range(0, len(i) - 1):
        distance += costs[tuple(i[j: j + 2])]
    tour_distances.append(distance)

print(tour_distances)
print(sum(tour_distances))

In [ ]:
tour_times = []

for i in tours:
    time = 0
    for j in range(0, len(i) - 1):
        time += costs[tuple(i[j: j + 2])] / v
    for j in range(0, len(i)):
        if i[j] in C:
            time += service_time_customer
        if i[j] in R:
            time += service_time_rs / 2
    tour_times.append(time)

print(tour_times)
print(sum(tour_times))

In [ ]:
api_file = open("apikey.txt", "r")
api_key = api_file.read()
api_file.close()

lats, lngs = zip(*locations_new)

import gmplot

gmap = gmplot.GoogleMapPlotter((north_bound + south_bound) / 2, (left_bound + right_bound) / 2, 13, apikey = api_key)

gmap.marker(north_bound, left_bound, color = 'gray')
gmap.marker(south_bound, left_bound, color = 'gray')
gmap.marker(north_bound, right_bound, color = 'gray')
gmap.marker(south_bound, right_bound, color = 'gray')

gmap.marker(lats[0], lngs[0], color = 'red')

gmap.scatter(lats[1: (nc + 1)], lngs[1: (nc + 1)], color = 'blue', size = 25, marker = False)

for i in range(0, nr):
    gmap.marker(lats[1 + nc + i], lngs[1 + nc + i], color = 'gold')

import polyline
for i in range(0, len(tours)):
    for j in range(0, len(tours[i]) - 1):
        if (tours[i][j], tours[i][j+1]) not in E_R:
            pll = routes_mapped[(tours[i][j], tours[i][j+1])]['routes'][0]['overview_polyline']['points']
            path = zip(*polyline.decode(pll))
            gmap.plot(*path, edge_width = 3, color = 'green', alpha = 0.5)
        
for i in range(len(lats)):
    gmap.text(lats[i] - 0.0007, lngs[i], str(i))

url = 'map_' + fpath[fpath.rindex('/') + 1:] + '_CPLEX.html'

gmap.draw(url)

from IPython.display import IFrame

IFrame(url, width = 500, height = 500)

In [ ]:
from openpyxl import load_workbook

wb = load_workbook('Results.xlsx')
ws = wb.active
ws.append([fpath[fpath.rindex('/') + 1:] + '_CPLEX', sum(tour_times), sum(tour_distances), str(tours_demapped)])
wb.save('Results.xlsx')